In [4]:
from json import loads
import pandas as pd
import numpy as np

In [5]:
song_scraped= loads(open("artists_scraped.json").read())
BB = pd.read_csv("data/Hot Stuff.csv")

In [6]:
print("No. Total Artists:    ", len(set(BB["Performer"])))
print("No. Artists Scraped:  ", len(song_scraped))
print("No. Missing Artists:  ", len(set(BB["Performer"]) - set(song_scraped.keys())))

No. Total Artists:     9447
No. Artists Scraped:   4523
No. Missing Artists:   4924


### Examine Commonality Between The Missing Artists

In [7]:
from collections import Counter

In [8]:
missing_artists = list(set(BB["Performer"]) - set(song_scraped.keys()))
def get_count(ls):
    bag_of_artists_name = list()
    for i in ls:
        bag_of_artists_name += i.split(" ")
    name_counts = Counter(bag_of_artists_name)
    print(name_counts.most_common(20))
get_count(missing_artists)

[('Featuring', 1930), ('&', 1198), ('The', 952), ('And', 250), ('Lil', 181), ('With', 167), ('Wayne', 108), ('Brown', 90), ('Chris', 87), ('and', 84), ('Band', 84), ('Drake', 80), ('Nicki', 74), ('His', 71), ('Young', 70), ('Minaj', 69), ('Orchestra', 69), ('Big', 68), ('Kanye', 64), ('West', 61)]


In [9]:
missing_artists_reformat = missing_artists
for i in range(len(missing_artists)):
    if "Featuring" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("Featuring")[0].strip()
    if "Feat." in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("Feat.")[0].strip()
    if "featuring" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("featuring")[0].strip()
    if "&" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("&")[0].strip()
    if "and" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("and")[0].strip()
    if "And" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("And")[0].strip()
    if "With" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("With")[0].strip()
    if "with" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("with")[0].strip()
    if "(" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("(")[0].strip()
get_count(missing_artists_reformat)

[('The', 569), ('B', 71), ('Lil', 66), ('Brown', 41), ('David', 40), ('John', 39), ('DJ', 37), ('Chris', 36), ('Of', 36), ('Drake', 35), ('Wayne', 35), ('Joe', 35), ('Big', 35), ('Paul', 34), ('Bobby', 33), ('Young', 29), ('Johnny', 28), ('Kanye', 28), ('Sean', 27), ('Jimmy', 26)]


In [10]:
import requests
import re
from random import randrange
from bs4 import BeautifulSoup
from collections import defaultdict

In [ ]:
art_song_links = dict()
comm = r'<table style="padding:2px; border-spacing:0px; border:0px; width:100%">\n<tr>\n<td style="width:50%; vertical-align:top;"><b class="text-yellow">Song<\/b><\/td>\n<td style="width:50%; vertical-align:top;"><b class="text-yellow">in Album<\/b><\/td><td>&nbsp;<\/td>\n<td>&nbsp;<\/td>\n<\/tr>\n([\S\s]*)\n<\/table>'
missing_artists_2 = set()
count = 0
try:
    for i in range(0, len(missing_artists_reformat)):
        ex = missing_artists_reformat[i]
        songs_array = []; album_array = []; url_array = []
        url = "https://lyrics.az/{}/allsongs.html"
        resp = requests.get(url.format("-".join(ex.lower().split())))
        print("Cnt: ", count)
        count += 1
        m = re.search(comm, resp.text)
        if m:
            rows = m.group(1)
            soup = BeautifulSoup(rows, 'html.parser')
            for r in soup.find_all('tr'):
                find_link_album_song = r'<tr>\n<td style=\"vertical-align:top;\"><a href=\"(.*\.html)\"><div class=\"schemadiv\" itemscope=\"\" itemtype=\"http:\/\/schema\.org\/MusicRecording\">\n<span itemprop=\"name\">(.*)<\/span><\/div><\/a><\/td>\n<td style=\"vertical-align:top;\">\[(<a href=\".*\" style=\"color: #ffffff;\">(.*)<\/a>)?]<\/td>\n<td>\s<\/td>\n<td>\s<\/td>\n<\/tr>'
                #print("'{}'".format(str(r)))
                m = re.search(find_link_album_song, str(r))
                #print(m.group(1),m.group(2),m.group(4))
                songs_array.append(m.group(2))
                album_array.append(m.group(4))
                url_array.append("https://lyrics.az{}".format(m.group(1)))
            art_song_links[missing_artists[i]] = {"Song": songs_array, "Album": album_array, "Url": url_array}
        else:
            print(ex)
            missing_artists_2.add(missing_artists[i])
except:
    print("Stop at {}".format(count))

Cnt:  0
Cnt:  1
Big Dee Irwin
Cnt:  2
Cnt:  3
The Marc Tanner B
Cnt:  4
Arpeggio
Cnt:  5
Cnt:  6
Cnt:  7
Cnt:  8
Mary J. Blige
Cnt:  9
Cnt:  10
T.I.
Cnt:  11
Cnt:  12
Cnt:  13
Cnt:  14
Cnt:  15
Monty Kelly
Cnt:  16
Miss Abrams
Cnt:  17
Cnt:  18
Tom Kimmel
Cnt:  19
Cnt:  20
Cnt:  21
Cnt:  22
Chris Columbo Quintet
Cnt:  23
O.V. Wright
Cnt:  24
Cnt:  25
D.H.T.
Cnt:  26
2 Chainz x Gucci Mane x Quavo
Cnt:  27
Cnt:  28
Hank Williams Jr.
Cnt:  29
Dr. Dre
Cnt:  30
Cnt:  31
Tin Tin
Cnt:  32
Cnt:  33
Cnt:  34
Cnt:  35
Cnt:  36
Cnt:  37
Cnt:  38
Cnt:  39
George Fischoff
Cnt:  40
Cnt:  41
Cnt:  42
Kongas
Cnt:  43
The Poppies
Cnt:  44
will.i.am
Cnt:  45
Tammy Montgomery
Cnt:  46
Carol Lynn Townes
Cnt:  47
Cnt:  48
Crystal Mansion
Cnt:  49
Cnt:  50
Eric B.
Cnt:  51
Mag 7
Cnt:  52
The Graces
Cnt:  53
Lil' Duval
Cnt:  54
Cnt:  55
K'Naan
Cnt:  56
Lea Roberts
Cnt:  57
Cnt:  58
The Joe Cuba Sextet
Cnt:  59
Kenny G.
Cnt:  60
Jimmy Norman
Cnt:  61
Marilyn Sellars
Cnt:  62
Eternity's Children
Cnt:  63
Cnt: 

In [ ]:
len(art_song_links)